# OpenAI API Platform

https://platform.openai.com/


### Prompt HyperParameters


1. **`temperature`**
    https://blog.lukesalamone.com/posts/what-is-temperature/
    - **작동 방식**:  
      확률 분포를 **평탄화하거나 집중**시킵니다.
      - 낮은 값 (`0.1` 등): 높은 확률의 토큰만 선택하게 하여 응답이 더 **결정적**(예측 가능)입니다.
      - 높은 값 (`1.5` 등): 확률 분포를 평탄화하여 저확률 토큰도 선택할 가능성이 있어 응답이 더 **창의적**이고 다양합니다.
    
    - **적용 범위**:
      - 문장이 단조롭거나 응답이 너무 예상 가능할 때 다양성을 부여.
      - 확률 분포를 조정하는 가장 기본적인 매개변수.
    
    - **비유**:
      - 음식을 만들 때 **조미료의 양을 조정**하는 것처럼, 다양성을 직접적으로 조절.

2. **`top-p`** (또는 **nucleus sampling**)
    - **작동 방식**:  
      확률 분포에서 누적 확률이 특정 값(`p`) 이하가 되는 상위 토큰들만 고려합니다.  
      - 예: `top-p=0.9`라면, 전체 확률의 **90%를 차지하는 상위 토큰**들만 선택 후보로 유지.
      - 확률 분포의 **꼬리 부분**(즉, 낮은 확률 토큰)을 잘라내어 선택 가능성을 제한합니다.
    
    - **적용 범위**:
      - 확률이 높은 후보군에 집중하면서도 너무 단조로운 응답을 방지.
      - 매우 드문(낮은 확률) 선택지를 배제.
    
    - **비유**:
      - 음식에서 **상위 재료 90%만 골라서 요리**하는 것처럼, 유의미한 선택만 남김.
3. **`최대 길이(Max Length)`**
    - 생성되는 토큰 수의 최대 길이를 제한한다.
    - 길이를 제한하여 응답 품질과 비용을 제어한다.
4. **`중지 시퀀스(Stop Sequences)`**
    - 특정 문자열에서 응답 생성을 중단한다.
    - 응답의 구조와 길이를 제어할 수 있다.
5. **`빈도 패널티(Frequency Penalty)`**
    - 특정 단어가 자주 반복되지 않도록 페널티를 적용한다.
    - 빈도 패널티가 높을수록 단어가 다시 나타날 가능성이 낮아짐.
    - 더 많이 나타나는 토큰에 더 높은 패널티를 부여함으로써 모델의 응답에서 단어의 반복을 줄임
6. **`존재 패널티(Presence Penalty)`**
    - 특정 단어가 한 번이라도 등장하면 페널티를 적용한다.
    - 빈도 패널티와 달리, 반복된 토큰에 대한 패널티를 적용하지만, 모든 반복된 토큰에 대해 패널티가 동일하다.
        - 두 번 나타나는 토큰과 열 번 나타나는 토큰이 같은 패널티를 받는다.
    - 모델이 응답에서 구절을 너무 자주 반복하는 것을 방지
    - 모델에게 다양하거나 창의적인 텍스트를 생성하도록 하고 싶다면 높은 존재 패널티를 사용


**temperature vs. top-p**

| **특징**           | **temperature**                        | **top-p**                            |
|---------------------|----------------------------------------|--------------------------------------|
| **작동 방식**       | 확률 분포를 평탄화하거나 집중          | 누적 확률 기준으로 후보 제한         |
| **결과 다양성**     | 전체 분포에서 다양성을 조정            | 높은 확률 토큰만 사용해 제어          |
| **조정 방식**       | 분포의 "모양"을 변경                  | 분포의 "범위"를 제한                 |
| **활용 사례**       | 모델의 창의성이나 결정성 직접 제어     | 비현실적 선택 방지                   |

**`temperature`와 `top-p`는 함께 결합해 사용할 수 있다**:
- `temperature=0.7, top-p=0.9`
- `temperature`로 창의성을 설정하고, `top-p`로 지나치게 낮은 확률의 토큰을 배제하여 모델이 **실용적이면서도 창의적**인 결과를 생성하게 만들수 있다.
  

# 프롬프트 엔지니어링 기법 모음

1. **Zero-shot, One-shot, Few-shot Prompting**
모델에게 제공하는 예시(Example)의 개수에 따라 구분하는 가장 기초적인 기법이다. 예시를 통해 모델이 수행해야 할 작업의 패턴과 답변 형식을 학습하도록 유도한다.
```text
[Zero-shot: 예시 없음]
"이 문장을 긍정/부정으로 분류하라: '오늘 날씨가 참 좋다.'"

[One-shot: 예시 1개]
"예시: '맛이 없다' -> 부정
문제: '오늘 날씨가 참 좋다' -> ?"

[Few-shot: 예시 여러 개]
"예시1: '맛이 없다' -> 부정
예시2: '최고의 영화다' -> 긍정
예시3: '다시 오고 싶지 않다' -> 부정
문제: '오늘 날씨가 참 좋다' -> ?"

```


2. **Chain-of-Thought (CoT)**
중간 추론 과정을 단계별로 명시하도록 유도한다.
```text
문제: 345와 678을 더한 뒤 12로 나눈 나머지를 구하라.
지시: 단계별로 생각 과정을 자세히 기술한 후 최종 답을 제시하라. “Let’s think step by step.”

```


3. **Tree-of-Thought (ToT)**
여러 사고 경로를 동시에 탐색하고 비교하도록 유도한다.
```text
문제: 한 변의 길이가 5인 정삼각형의 넓이를 구하라.
지시:
1) 방법 A: 공식 사용
2) 방법 B: 좌표평면에 넣어 계산
두 경로를 각각 전개한 뒤, 결과와 장단점을 비교·선택하라.

```


4. **Self-Consistency**
여러 번 답안을 생성한 뒤, 가장 빈번하게 등장한 답을 선택하도록 유도한다.
```text
 문제: “racecar”가 팰린드롬인지 판별하라.
 지시: 이 질문을 5회 반복적으로 답변한 뒤, 가장 일관된 답을 최종으로 제시하라.

```


5. **Least-to-Most**
문제를 작은 단계로 나눈 뒤, 쉬운 것부터 차례로 해결하도록 유도한다.
```text
문제: “1234 + 5678 − 910” 계산하라.
지시:
1) 먼저 1234 + 5678을 계산하라.
2) 그 결과에서 910을 빼라.
3) 각 단계의 답을 제시하고, 최종 결과를 계산하라.

```


6. **Reflection**
**Reflection(자기 성찰)**은 AI가 내놓은 첫 번째 답변을 스스로 다시 읽어보고, **논리적 허점이나 오류를 찾아내어 스스로 수정**하도록 유도한다. `생성(Generation) - 비판(Critique)- 개선(Refinement)` 과정을 거친다.
```text
[1단계: 초안 작성]
"2026년 이커머스 트렌드에 대한 기획안 초안을 작성해라."
[2단계: 자기 비판]
"방금 작성한 기획안을 비판적인 전문가의 시각에서 검토해라. 논리적 근거가 부족한 부분이나 실현 가능성이 낮은 제안 3가지를 찾아내라."
[3단계: 최종 수정]
"찾아낸 문제점들을 보완하여 데이터 중심의 구체적인 해결책이 포함된 최종 기획안으로 수정해라."

```


7. **Plan-and-Solve (PS)**
전체 풀이 계획을 먼저 작성하고, 그 계획에 따라 차례로 풀이하도록 유도한다.
```text
문제: “2차 방정식 x^2 − 5x + 6 = 0의 해를 구하라.”
지시:
1) 계획 세우기: 필요한 공식·단계를 나열하라.
2) 계획에 따라 단계별로 해를 구하라.

```


8. **ReAct (Reasoning and Acting)**
추론과 외부 행동(e.g., 검색, 코드 실행)을 번갈아가며 수행하도록 유도한다.
```text
문제: “2025년 FIFA 여자 월드컵 개최국은 어디인가?”
지시:
- Reasoning: 관련 지식을 확인한 뒤
- Action: “search(‘2025 FIFA Women’s World Cup host country’)” 명령어를 사용해 인터넷에서 최신 정보를 가져오라
- Reasoning: 검색 결과를 바탕으로 최종 답을 제시하라.

```



### 실습예제

#### 1. Zero-shot Prompting (제로샷)

모델에게 아무런 예시도 제공하지 않고, 지시사항(Instruction)만으로 답변을 요구하는 방식이다. 모델이 사전 학습(Pre-training)된 지식에 전적으로 의존한다.

* **System Prompt:**

```text
너는 감정을 분석하는 AI이다. 주어진 문장의 감정을 '긍정', '부정', '중립' 중 하나로만 분류하여 대답하라.

```

* **User Prompt:**

```text
문장: "이 식당은 맛은 훌륭하지만, 서비스가 너무 느려서 다시 가고 싶지 않아."
감정:

문장: "너무 맛있지만, 조미료를 많이 썼는지 배아팠어요"
감정:

문장: "존맛탱"
감정:

```

* **권장 하이퍼파라미터:** `Temperature: 0.2`, `Top_p: 0.9` (분류 작업이므로 일관성이 중요함)



#### 2. One-shot Prompting (원샷)

모델에게 **단 하나의 예시(입력-출력 쌍)**를 제공하여, 원하는 답변의 형식이나 톤앤매너(Tone & Manner)를 가이드하는 방식이다.

* **System Prompt:**

```text
너는 영어를 한국어로 번역하는 번역가이다. 단, 예시와 같이 고전적인 '사극(역사 드라마)' 말투로 번역해야 한다.

```

* **User Prompt:**

```text
예시:
Input: "I will never forgive you."
Output: "내 그대를 결코 용서치 않을 것이오."

문제:
Input: "Please bring me some water."
Output:

```

* **권장 하이퍼파라미터:** `Temperature: 0.7`, `Top_p: 0.9` (말투의 뉘앙스를 살리기 위해 약간의 창의성 허용)


#### 3. Few-shot Prompting (퓨샷)

모델에게 **2개 이상의 예시**를 제공하여 복잡한 패턴, 규칙, 또는 뉘앙스를 학습시키는 방식이다. 모델이 문맥을 통해 규칙을 추론하게 만든다. (In-context Learning)

* **System Prompt:**

```text
너는 고객의 리뷰를 분석하여, 그들이 가장 중요하게 생각하는 '핵심 키워드'를 추출하는 봇이다. 아래 예시들의 패턴을 학습하여 답변하라.

```

* **User Prompt:**

```text
검토 1: "배송이 정말 빨라서 놀랐어요. 근데 포장이 좀 뜯겨 있네요."
핵심: 배송 속도 빠름, 포장 상태 불량

검토 2: "디자인은 예쁜데, 마감이 너무 허술해서 실망입니다. 가격 대비 별로예요."
핵심: 디자인 좋음, 마감 품질 허술, 가성비 별로

검토 3: "소음이 거의 없어서 밤에 쓰기 딱 좋아요. 다만 전원 선이 좀 짧습니다."
핵심: 소음 거의 없음, 전원 선 길이 짧음

검토 4: "화면 색감이 쨍하고 선명해서 영상 볼 맛이 나네요. 무게도 가벼워요."
핵심:

```

* **권장 하이퍼파라미터:** `Temperature: 0.3`, `Top_p: 0.9` (예시의 패턴을 충실히 따라야 하므로 낮은 무작위성 권장)


#### 4. Chain-of-Thought (CoT)

논리적 추론이 필요한 수학 문제나 수수께끼를 통해 AI가 과정을 설명하도록 유도하는 테스트이다.

* **System Prompt:**

```text
너는 문제를 해결할 때 논리적인 사고의 흐름을 중시하는 수학 튜터이다. 정답을 바로 말하지 말고, 도출하기까지의 과정을 단계별로 상세히 설명해야 한다.

```

* **User Prompt:**

```text
어떤 농장에 닭과 소가 있다. 머리의 수는 총 20개이고, 다리의 수는 총 56개이다. 농장에 있는 닭과 소는 각각 몇 마리인가? "Let's think step by step"을 사용하여 풀이하라.

```

* **권장 하이퍼파라미터:** `Temperature: 0.2`, `Top_p: 0.9` (논리적 정확성이 중요하므로 무작위성을 낮춤)


#### 5. Tree-of-Thought (ToT)

단순한 답이 아니라, 여러 가능성을 열어두고 비교 분석해야 하는 문제 해결 시나리오이다.

* **System Prompt:**

```text
너는 전략 컨설턴트이다. 하나의 문제에 대해 서로 다른 3가지 접근 방식을 제안하고, 각 방식의 장단점을 시뮬레이션한 뒤 최적의 결론을 도출해야 한다.

```

* **User Prompt:**

```text
문제: 한정된 예산(100만 원)으로 일주일 안에 회사 브랜드를 홍보해야 한다.
지시:
1) 소셜 미디어 광고 집중
2) 인플루언서 협찬
3) 게릴라 오프라인 이벤트
위 3가지 경로(Path)에 대해 각각의 기대 효과와 리스크를 전개한 뒤, 가장 효율적인 방안 하나를 선정하라.

```

* **권장 하이퍼파라미터:** `Temperature: 0.7`, `Top_p: 0.95` (다양한 사고 확장을 위해 창의성을 다소 높임)


#### 6. Self-Consistency

AI가 할루시네이션(거짓 정보)을 범하기 쉬운 문제에서 다수결 원칙을 통해 정답을 찾는 테스트이다. (API가 아닌 챗봇 환경 실습용으로 변형함)

* **System Prompt:**

```text
너는 매우 신중한 검토자이다. 실수할 가능성이 있는 문제에 대해 여러 번 검산을 수행하고, 가장 확실한 답을 내놓아야 한다.

```

* **User Prompt:**

```text
문제: 내가 사과 5개를 가지고 있다. 어제 2개를 먹었고, 오늘 친구가 3개를 더 주었다. 그 후 내가 1개를 잃어버렸다면, 지금 가진 사과는 몇 개인지 계산하라.
지시: 이 계산을 서로 다른 문장 구조로 3번 반복하여 풀이한 뒤, 3번의 결과 중 가장 많이 일치하는 답을 최종 정답으로 제시하라.

```

* **권장 하이퍼파라미터:** `Temperature: 0.5`, `Top_p: 0.9` (다양한 답변 생성을 위해 약간의 변동성을 주되, 논리는 유지함)


#### 7. Least-to-Most

한 번에 풀기 어려운 복잡한 언어 처리 작업을 분해하여 해결하는 테스트이다.

* **System Prompt:**

```text
너는 복잡한 지시사항을 수행할 때, 이를 작은 단위로 쪼개어 순차적으로 처리하는 언어 분석가이다.

```

* **User Prompt:**

```text
문제: "Artificial Intelligence"라는 단어에서 모음(a, e, i, o, u)만 추출한 뒤, 그 추출된 글자들을 알파벳 역순(z-a)으로 정렬하여 하나의 문자열로 만들어라.
지시:
1) 원본 단어에서 모음을 모두 찾아 나열하라.
2) 찾은 모음들을 알파벳 역순으로 정렬하라.
3) 최종 결과 문자열을 제시하라.

```

* **권장 하이퍼파라미터:** `Temperature: 0.1`, `Top_p: 0.1` (지시 이행의 정확도가 최우선이므로 매우 낮게 설정)


#### 8. Reflection

AI가 작성한 초안을 스스로 비판하고 수정하여 결과물의 품질을 높이는 테스트이다.

* **System Prompt:**

```text
너는 글쓰기 작가이자 까다로운 편집자이다. 네가 쓴 글을 스스로 비판적으로 검토하고 더 나은 방향으로 고칠 수 있는 능력이 있다.

```

* **User Prompt:**

```text
1. [초안 작성]: 고객에게 보내는 '서비스 요금 인상 안내' 이메일 초안을 정중하게 작성하라.
2. [자기 비판]: 위 초안에서 고객이 불쾌감을 느낄 수 있는 부분이나, 설명이 부족한 부분을 2가지 지적하라.
3. [최종 수정]: 지적된 내용을 바탕으로 고객을 설득할 수 있도록 이메일을 수정하여 완성하라.

```

* **권장 하이퍼파라미터:** `Temperature: 0.6`, `Top_p: 0.9` (초안 작성과 수정 과정에서 적절한 자연스러움이 필요함)


#### 9. Plan-and-Solve (PS)

과학 문제나 알고리즘 문제처럼 절차가 중요한 영역에서 계획 수립 능력을 테스트한다.

* **System Prompt:**

```text
너는 문제를 받으면 무작정 풀지 않고, 명확한 계획(Plan)을 먼저 세운 뒤 실행(Solve)하는 엔지니어이다.

```

* **User Prompt:**

```text
문제: 시속 60km로 달리는 기차가 길이 1.2km의 터널을 완전히 통과하는 데 1분 30초가 걸렸다. 이 기차의 길이를 구하라.
지시:
1) [Plan]: 단위를 통일하고 사용할 공식을 정의하라.
2) [Solve]: 계획에 따라 값을 대입하여 기차의 길이를 계산하라.

```

* **권장 하이퍼파라미터:** `Temperature: 0.1`, `Top_p: 0.5` (수식과 계산의 정확성이 중요함)


#### 10. ReAct (Reasoning and Acting)

> 26/01/07 현재 Playground Chat - Response API의 웹검색 도구의 성능은 좋지 않다. 엉뚱한 소스를 찾아 참조함.

외부 정보 검색이나 도구 사용이 필요한 최신 정보 관련 질문 테스트이다. (웹 검색 기능이 있는 모델에서 유효함)

* **System Prompt:**

```text
너는 지식의 한계를 인지하고, 필요한 경우 외부 검색 도구를 활용하여 사실관계를 검증하는 연구원이다. 생각(Reasoning), 행동(Action), 관찰(Observation)의 과정을 따르라.

```

* **User Prompt:**

```text
문제: 2024년 아카데미 시상식(오스카)에서 작품상을 받은 영화의 감독은 누구인가?
지시:
- Reasoning: 내가 알고 있는 지식인지 판단한다.
- Action: 최신 정보 확인을 위해 검색 쿼리를 생성한다. (예: search("2024 Oscars Best Picture winner director"))
- Reasoning: 검색 결과를 바탕으로 최종 답을 도출한다.

```

* **권장 하이퍼파라미터:** `Temperature: 0.2`, `Top_p: 0.8` (사실 정보 검색이므로 환각을 최소화해야 함)